In [ ]:
#this query takes existing antihypertensive medication table for three different 
#time points and unions them together to put it into 
#long format so that analysis can be performed in jupyterlab. updated for social care and primary care data

create or replace table `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_antihypertensive_long_format` as (
with g as(select
 a.person_id,
 b.*except(person_id),
 'post_admission' as time_status
 from `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_hypertension_combined` a 
 full join `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_antihypertensive_post_admission` b on a.person_id = b.person_id
 where a.hypertension = TRUE

 union all

select
 c.person_id,
 d.*except(person_id),
 'admission' as time_status
 from `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_hypertension_combined` c
full join `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_antihypertensive_admission` d on c.person_id = d.person_id
 where c.hypertension = true

union all 

select
 e.person_id,
 f.*except(person_id),
 'pre_admission' as time_status
 from `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_hypertension_combined` e
full join `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_antihypertensive_pre_admission` f on e.person_id = f.person_id
 where e.hypertension = true),

h as (
select
person_id,
ifnull(beta_blocker, 'false') as beta_blocker,
ifnull(calcium_channel_blocker, 'false') as calcium_channel_blocker,
ifnull(thiazide_type_diuretic, 'false') as thiazide_type_diuretic,
ifnull(angiotensin_receptor_blocker, 'false') as angiotensin_receptor_blocker,
ifnull(ace_inhibitor, 'false') as ace_inhibitor,
ifnull(alpha_blocker, 'false') as alpha_blocker,
ifnull(no_antihypertensive,0) as no_antihypertensive,
time_status
from g
where person_id is not null
order by person_id, time_status)
#this restricts analysis to those who did not die within 6 months
SELECT
 h.*, 
 case when j.dod < datetime_add(j.date_of_admission,INTERVAL 6 MONTH) then 'true' else 'false' end as early_death
FROM 
h
left join 
`yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` j
on h.person_id = j.person_id)
